## PnL Explain :  Estimating PnL using sensitivities and Market Data

### Introduction

The PnL explain technique seeks to estimate the daily PnL from the change in the underlying risk factors.
In this case the risk factors are the yield curve rates for each tenor with a yield curve.

Usually a portfolio risk manager will monitor the risk factors that impact his portfolio, instead of monitoring all the positions booked in his portfolio. To assess what will be the value of his portfolio, a portfolio risk manager is interested in performing what if analysis based on a scenario of the risk factor values.

In this notebook, we perform a simplified PnL Explained by using Delta(Δ) to represent our sensitivity instead of the full Greek. We will utilize the various features of Atoti libraries to:

- load data into a multi-dimensional cube

* Explore the Data using the embedded visualization or Atoti UI
* Calculate Estimated PnL using sensitivities and Market Data
* Run multiple scenarios of the Yield Curve Stress Test

In [ ]:
import atoti as tt
import wget

In [ ]:
current_version = tt.__version__
used_version = '0.3.0.20200325172102'

# In the event a newer version of Atoti is used, please update the necessary functions according to the changelog
if used_version != current_version:
    raise ValueError('Version mismatch')

In [ ]:
#tell atoti to load the database containing the UI dashboards
config = tt.config.create_config(metadata_db="./metadata.db")

# you can pass in the parameter sampling_mode=tt.sampling.FULL to change the default sampling mode
# a session has to be created for atoti
session = tt.create_session(config=config, sampling_mode=tt.sampling.FULL)

## 1.1 Loading Data and creating ActivePivot multidimensional environment
### Creating Atoti DataStore

In [ ]:
# downloading data files for this notebook - skip if you already have
wget.download('http://data.atoti.io/notebooks/pnl-explained/position_sensitivities.csv')
wget.download('http://data.atoti.io/notebooks/pnl-explained/position_data.csv')
wget.download('http://data.atoti.io/notebooks/pnl-explained/trading_desk.csv')
wget.download('http://data.atoti.io/notebooks/pnl-explained/market_data.csv')

There are many ways Atoti can [consume data](https://www.atoti.io/documentation/tutorial/08-data-sources.html). In this notebook, we will be using read_csv to load data into the datastores.

### Position Sensitivities

In [ ]:
position_sensitivity_store = session.read_csv('position_sensitivities.csv', 
                                            keys=['book_id', 'instrument_code', 'currency', 'curve', 'tenor'], 
                                            store_name="Position Sensitivities Store")

In [ ]:
# you can use head(n) to view the first n rows of the store.
# likewise, you can use position_sensitivity_store.columns to view the columns available in the store
# lastly, you can use position_sensitivity_store.shape to view the number of rows and columns loaded into the store
position_sensitivity_store.head(5)

### Position Data

In [ ]:
position_store = session.read_csv('position_data.csv', 
                                            keys=['book_id', 'instrument_code'], 
                                            store_name="Position Store")

In [ ]:
position_store.head(5)

### Portfolio structure

In [ ]:
trading_desk_store = session.read_csv('trading_desk.csv', keys=['book_id'], store_name="Trading Desk")

In [ ]:
trading_desk_store.head(5)

### Market data

In [ ]:
market_data_store = session.read_csv('market_data.csv', keys=['currency', 'curve','tenor'], store_name="Market Data")

In [ ]:
market_data_store.head()

### Creating references between stores  

We will proceed to set up references between the stores that we just created as follows:
<img src="http://data.atoti.io/notebooks/pnl-explained/datastore_relations.png" alt="datastore relationship" style="zoom:40%;" />
  
We can see that all the stores are joined to *Position Sensitivities Store*, this is what we call the base store.  
For a record to be reachable in the cube, it must exists in the base store.  
We will perform the join from position_sensitivity_store.

In [ ]:
position_sensitivity_store.join(trading_desk_store, mapping={'book_id' : 'book_id'})

In [ ]:
position_sensitivity_store.join(position_store, mapping={'book_id' : 'book_id', 'instrument_code' : 'instrument_code'})

In [ ]:
position_sensitivity_store.join(market_data_store, mapping={'currency' : 'currency', 'curve':'curve', 'tenor':'tenor'})

In [ ]:
# using joined_columns=True allows you to preview the joins created
position_sensitivity_store.head(joined_columns=True)

### Creating cube  
We create the cube using the base store *position_sensitivity_store*.  
Note that we haven't pass in any [mode](https://www.atoti.io/documentation/lib/atoti.html#atoti.session.Session.create_cube) in create_cube(). This meant that a hierarchy will be automatically created for each non-numeric column and measure will be automatically created for each numeric column.  

In [ ]:
position_sensitivity_cube = session.create_cube(position_sensitivity_store, "Position_Sensitivities")

### Explore the Data Set as a Cube

We have the option to visualize the cube in chart, feature-value, pivot-table or tabular.  
Let's look at the sensitivity across currency for each asset class.

In [ ]:
position_sensitivity_cube.visualize()

### Adding business logic calculation  
  
Let's assign a variable to the attributes of the cube, so that we can:  
* create measures
* create hierarchies 

In [ ]:
m = position_sensitivity_cube.measures
h = position_sensitivity_cube.hierarchies
lvl = position_sensitivity_cube.levels

In [ ]:
# inspect what hierarchies have been automatically generated during cube creation
h

In [ ]:
# inspect what measures have been automatically generated during cube creation
m

#### Creating Measure

From the *Market Data* store, we have the *start of day* and *end of day* value which we used to calculated the change in yields. 
We derive our Delta by applying `sensi.SUM` to the notional. We, then apply Delta to our change in yield to get the impact of prices:
  
$Theoretical Pnl = (Delta) \times (Yield Change)$  

We will aggregate the *Theoretical PnL* over the levels listed in the scope below as we will be exploring the measure over these levels.

In [ ]:
m['Theoretical PnL'] = tt.agg.sum(m['sensi.SUM'] * (m['last.VALUE']-m['start_of_day.VALUE']) * m['notional.SUM'], scope=tt.scope.origin('currency', 'curve', 'tenor', 'book_id', 'instrument_code'))

### Adding new multi-level hierarchy for portfolio structure  

Based on the data that are of interest to the Buy-Side and Sell-Side, we are going to see how we can structure hierarchies to facilitate the navigation of data in a cube.  

#### Asset Mangement : Buy-Side  

*Buy-Side* purchases stocks, securities and other financial products based on the needs and strategy of a portfolio's need.  
It would make data navigation more intuitive by creating an *Investment Portfolio Hierarchy* that has Asset Class, Sub Asset Class, Fund and Portfolio as levels.  
We would be able to have a global view over the Asset Class level. If we need more granular information, we can easily drill-down to the Sub Asset class, Fund and all the way down to the portfolio holding the Asset.

In [ ]:
h["Investment Portfolio Hierarchy"] = {
    'Asset Class':position_sensitivity_cube.levels['asset_class'],
    'Sub Asset Class':position_sensitivity_cube.levels['sub_asset_class'],
    'Fund' : position_sensitivity_cube.levels['fund'], 
    'Portfolio':position_sensitivity_cube.levels['portfolio']
}

#### Investment Banks : Sell-Side  

*Sell-Side* help companies to raise dept and equity capital and then sell those securities to the *Buy-Side*.  \
*Sell-Side* would then be interested in having a global view over the *Business Unit* such as Rates & Credit, Forex, Equity etc.    
They could then drill-down to the *Sub Business Unit* to see its performance, the *Trading Desk* and all the way to the *Book* level.

In [ ]:
h["Trading Book Hierarchy"] = {
    'Business Unit':lvl['business_unit'],
    'Sub Business Unit':lvl['sub_business_unit'],
    'Trading Desk' : lvl['trading_desk'], 
    'Book':lvl['book']
}

### Explore the Theoretical PnL by Investment Portfolio Hierarchy

We shall explore the Theoretical PnL from the Buy-Side perspective.  
  
Let's visualize the data in a chart to see the spread of the *Theoretical PnL* across the funds for each *Asset Class*.

In [ ]:
position_sensitivity_cube.visualize("Theoretical PnL spread across funds")

In the next 2 visualizations, we shall see the impact of having the *Investment Portfolio Hierarchy*.

In the first tree map, we perform a split at the *portfolio* level from *Investment Portfolio Hierarchy*. This means that we are drilling down from the Asset class to Sub Asset Class, Fund and then to the Portfolio level. E.g. we will see the portfolio HE01 under the Asset Class *Rates & Credit* and *Forex*.

In the second tree map, we perform a split at the *portfolio* hierarchy. Hence we will only see the collective *Theoretical PnL* of the portfolio. E.g. we will only see 1 HE01 in this map.

In [ ]:
position_sensitivity_cube.visualize("Investment Portfolio concentration")

In [ ]:
position_sensitivity_cube.visualize("Portfolio concentration")

In the below pivot-table, we can easily drill-down the levels in the *Investment Portfolio Hierarchy* to see measures at granular levels.  
Naturally, this could also be achieved by clicking on `>+` and manually adding a hierarchy to drill down to. It's just a little more tedious.

In [ ]:
position_sensitivity_cube.visualize('Investment Portfolio Hierarchy Pivot Table')

#### Yield Curve

The *Yield Curve* gives insights to the future interest rate changes and economic activity. 
  
A normal yield curve which shows an upward curve, shows longer-term bonds having higher yields than short-term ones. Short-term interest rates are the lowest because there is less embedded inflation risk. This shows economic expansion.

An inverted curve which has a downward slope, is a sign of an upcoming recession. The shorter-term bonds yields higher than the longer-term ones.  

A flat or humped yield curve shows that the yields for shorter- and longer-term bonds are very close to each other. Investors are expecting interest rates to remain about the same, probably an economic transition.
  
We will use the start of day and last rates against the tenor to plot our yield curve. We should be able to see a normal upward yield curve in the chart below.

In [ ]:
position_sensitivity_cube.visualize('Yield Curve')

We see that *Rates & Credit* has the highest *Theoretical PnL* among the Asset class.

In [ ]:
position_sensitivity_cube.visualize('Theoretical PnL')

### What-ifs  
We will run simulations to see the impact of shifts in curves: 
- Parallel Shift
- Curve Inversion
- Curve Inversion Stress


By applying sensitivity against the Asset Class and Currency, we can see that the 3 peaks are in the order of *Rates & Credit, EUR*, *Forex, EUR* and lastly *Equity, EUR*. 

In [ ]:
position_sensitivity_cube.visualize("Risk Map")

Let's start by setting up a simulation based on *tenor* and *currency*, that fluctuates the *last.VALUE*. 
We will label the curve derived from the original cube data as *Last Curve*

In [ ]:
curve_simulation = position_sensitivity_cube.setup_simulation("Curve Simulation", per=[lvl["tenor"],lvl["currency"]], add=[m["last.VALUE"]], base_scenario_name = 'Last Curve')

#### Shift Simulation  
  
A parallel shift in the yield curve happens when the interest rates on all fixed-income maturities increase or decrease by the same number of basis point.  
The curve does not change but it shifts to the left or to the right. This is most common when the yield curve is upward sloping.  
  
This simulation is important for investors who might liquidate their positions before maturity, as the shift can cause bond prices to fluctuate substantially.  
Investors could mitigate this risk by reducing the bond duration, alleviate the volatility.
  
We will simulate parallel shift in yield curve by applying a negative 10 bps shift on the last rate for Euro currency on all Tenor.

In [ ]:
# create the scenario
curve_shift = curve_simulation.scenarios['Curve Parallel Shift']

In [ ]:
# state the condition of the scenario. -0.001 will be added to the last.VALUE for EUR
curve_shift += ("*", "EUR", -0.001, tt.simulation.Priority.CRITICAL)

In [ ]:
# preview the condition of the simulation
curve_shift.head(5)

##### Explore Curve Parallel Shift scenario impact on Theoretical PnL  
  
We see that while the curve retains its shape, all the data points moved to the right of the graph.

In [ ]:
position_sensitivity_cube.visualize('Yield Curve Curve Parallel Shift')

We go into the investment portfolio to see the impact of this shift on the *Theoretical PnL*.  \
We see the *Theoretical PnL* went negative on a 10bps shift, with *Rates & Credit* suffering the most loss.  

In [ ]:
position_sensitivity_cube.visualize('Theoretical PnL Parallel Shift - Investment Portfolio')

#### Curve Inversion Simulation

An inverted yield curve is a predictor of economic recession as it implies that interest rates are going to fall. In fact, recessions usually cause interest rates to fall.  
  
Short-term bills are expected to plummet if recession is coming, as Federal Reserve will lower the fed funds rate when economy slows down. Therefore investors may avoid Treasurys with maturities of less than two years to have a safe investment. This sets back the demand for those bills and sends their yields up. Higher demands in the longer-term instruments lower the yields, hence an inverted curve occurs.

Let's run a simulation on the currency EUR and see the impact on the *Theoretical PnL*.
We will set a negative 20bps on `last.VALUE` for the currency EUR for tenors 5Y and above and observe the change in the curve shape.

In [ ]:
curve_inversion = curve_simulation.scenarios['Curve Inversion']

In [ ]:
curve_inversion += ("5Y", "EUR", -0.002, tt.simulation.Priority.CRITICAL)
curve_inversion += ("6Y", "EUR", -0.002, tt.simulation.Priority.CRITICAL)
curve_inversion += ("7Y", "EUR", -0.002, tt.simulation.Priority.CRITICAL)
curve_inversion += ("8Y", "EUR", -0.002, tt.simulation.Priority.CRITICAL)
curve_inversion += ("9Y", "EUR", -0.002, tt.simulation.Priority.CRITICAL)

In [ ]:
curve_inversion.head(5)

##### Explore Curve Inversion scenario impact on Theoretical PnL  
  
We see a slight inversion in the curve from the tenor 4Y to 5Y before it becomes a shift to the right, as in the case of a parallel shift.  
In this case, we predict that the yields will dip for instruments with more than 5Y maturity.

In [ ]:
position_sensitivity_cube.visualize('Yield Curve Inversion')

As expected, we can see drastic drop in the *Theoretical PnL* across all the asset classes, the greatest loss incurred in the asset class *Rates & Credit*.

In [ ]:
position_sensitivity_cube.visualize('Theoretical PnL Curve Inversion')

In [ ]:
position_sensitivity_cube.visualize("Theoretical PnL against Tenor")

#### Curve Inversion Stress Simulation  
  
We will create a new scenario where we stress the curve further by assuming further drops in rates for the longer-term instruments.  

In [ ]:
curve_inversion_stress = curve_simulation.scenarios['Curve Inversion Stress']

In [ ]:
curve_inversion_stress += ("5Y", "EUR", -0.002, tt.simulation.Priority.CRITICAL)
curve_inversion_stress += ("6Y", "EUR", -0.002, tt.simulation.Priority.CRITICAL)
curve_inversion_stress += ("7Y", "EUR", -0.004, tt.simulation.Priority.CRITICAL)
curve_inversion_stress += ("8Y", "EUR", -0.004, tt.simulation.Priority.CRITICAL)
curve_inversion_stress += ("9Y", "EUR", -0.004, tt.simulation.Priority.CRITICAL)

In [ ]:
curve_inversion_stress.head(5)

##### Explore Curve Inversion Stress scenario impact on Theoretical PnL  
  
We see the curve is starting to slope more downward.

In [ ]:
position_sensitivity_cube.visualize('Yield Curve Inversion Stress')

We see the downward slope dipped further in the stress scenario.

In [ ]:
position_sensitivity_cube.visualize('Theoretical PnL Curve Inversion Stress')

In [ ]:
position_sensitivity_cube.visualize('Scenarios Comparison')

In [ ]:
session.url

### Build Your Standalone App using ActiveUI:

* Publish Yield Curve and PnL Views
* Add Page Quick filters
* Compare scenarios
* Save Dashboards

We can publish the visualizations above as widgets to Atoti UI. Click on *Open App* when you have published all the widgets of interest.  

<img src="http://data.atoti.io/notebooks/pnl-explained/publish_widget.gif" alt="publish widget" style="zoom:40%;" />

We can quickly put together a PnL Explained dashboard.  

<img src="http://data.atoti.io/notebooks/pnl-explained/dashboard_design.gif" alt="dashboard design" style="zoom:40%;" />

In [ ]:
from IPython.display import Markdown as md 
md("[" + session.url + "/#/dashboard/680](" + session.url + "/#/dashboard/680)")